In [161]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import matplotlib.pyplot as plt



In [162]:
df = pd.read_csv(f'../data/market_integrates_100.csv')
print(df)


      market_ids  firm_ids  characteristic1  characteristic2     prices  \
0              1         1         5.741020         2.586439  38.778167   
1              1         2         5.728377         2.774003  37.705697   
2              1         3         5.158599         2.988436  41.650372   
3              1         4         5.159242         2.188203  39.860431   
4              1         5         5.664929         2.845616  39.833859   
...          ...       ...              ...              ...        ...   
1995         100        16         5.547604         2.593617  48.170051   
1996         100        17         5.278139         2.755727  56.017861   
1997         100        18         5.668548         2.623788  46.831919   
1998         100        19         5.336720         2.218909  47.785429   
1999         100        20         5.495126         2.636772  49.415789   

      marginal_cost    shares   profits   markups  e_quantity    capital  \
0         30.104214  0.

In [163]:
# theta_0 = np.array([2, -0.5, 0.3, 0.05, 1.])
theta_0 = np.array([1., -1., 1., 0.5, 1.])




In [164]:
def simulate_market_shares_per_period(theta, df, v_p, n_consumers, n_firms, t): 
    beta = theta[:3]
    mu = theta[3]
    omega = theta[4]
    X0 = np.ones(n_firms)
    X1 = df.characteristic1.head(n_firms)
    X2 = df.characteristic2.head(n_firms)
    X_car = np.column_stack((X1, X2))
    X = np.column_stack((X0, X_car))

    X_for_utility = np.repeat(X, n_consumers, axis=0)
    price_r = np.reshape(np.array(df.prices[t*n_firms:(t+1)*n_firms]), (1, n_firms))
    alpha_i = np.reshape(-(np.exp(mu + omega*v_p)), (n_consumers, 1))
    random_coeff = np.ravel((alpha_i*price_r).T)

    u = X_for_utility@beta + random_coeff
    u_r = np.reshape(u, (n_firms, n_consumers))
    sum_u = np.sum(np.exp(u_r))

    all_probs = np.exp(u_r)/(1 + sum_u)
    market_shares = np.sum(all_probs, axis=1)
    
    return market_shares

In [165]:
def f(theta, df, n_consumers, n_firms, T):
    true_share = df.shares
    all_est_shares = np.zeros(T*n_firms)
    v_p_all = np.random.normal(0, 1, n_consumers*T)
    for t in range(0, T-1):
        v_p = v_p_all[t*n_consumers:(t+1)*n_consumers]
        estimates_shares_market = simulate_market_shares_per_period(theta, df, v_p, n_consumers, n_firms, t)
        all_est_shares[t*n_firms:(t+1)*n_firms] = estimates_shares_market
    return np.linalg.norm(true_share - all_est_shares)

In [168]:
N = 1000 
firms  = 20
time = 100 

all_val_b0 = []
all_val_b1 = []
all_val_b2 = []
all_val_mu = []
all_val_omega = []

for l in range(1, 2):
    
    res = minimize(f, theta_0, args=(df, N, firms, time), method = 'Nelder-Mead')
    optim_vec = res.x

    all_val_b0.append(optim_vec[0])
    all_val_b1.append(optim_vec[1])
    all_val_b2.append(optim_vec[2])
    all_val_mu.append(optim_vec[3])
    all_val_omega.append(optim_vec[4])


df_estimation_values = pd.DataFrame({'beta_0': all_val_b0,
                                    'beta_1': all_val_b1,
                                    'beta_2': all_val_b2, 
                                    'mu': all_val_mu, 
                                    'omega': all_val_omega, 
                                    })
df_estimation_values.to_csv(f'../data/first_estimation_market_inter_100', index=False)


In [169]:
print(df_estimation_values)

     beta_0    beta_1    beta_2        mu     omega
0  1.001611 -0.836345  1.069995  0.503408  1.096109
